In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccsddl51',
    proc_date,
    'temp_sap_vccsddl51'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl51(    
    hash_id BIGINT,
    calday DATE,
    do_number STRING,
    dc_id STRING,
    store_id STRING,
    plan_gi_date DATE,
    actual_gi_dte DATE,
    proc_date DATE,
    file_creation_ts TIMESTAMP
)
TBLPROPERTIES 
(
	DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE = TRUE,
	DELTA.AUTOOPTIMIZE.AUTOCOMPACT = TRUE
)
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccsddl51 main_tbl 
USING (
    WITH main AS (
        SELECT 
            ROW_NUMBER() OVER (PARTITION BY deliv_numb ORDER BY reqtsn DESC, datapakid DESC, record DESC) rn,
            file_creation_ts,
            date_format(to_date(calday, 'yyyyMMdd'), 'yyyy-MM-dd') AS calday,
            deliv_numb AS do_number,
            ship_point AS dc_id,
            LTRIM('0', ship_to) AS store_id,
            date_format(to_date(gi_date, 'yyyyMMdd'), 'yyyy-MM-dd') AS plan_gi_date,
            date_format(to_date(act_gi_dte, 'yyyyMMdd'), 'yyyy-MM-dd') AS actual_gi_date,
            proc_date
        FROM (
            SELECT * EXCEPT(rn) FROM (
                SELECT *, ROW_NUMBER() OVER (
                    PARTITION BY reqtsn, datapakid, record 
                    ORDER BY proc_date DESC
                ) rn 
                FROM temp_sap_vccsddl51
            ) a WHERE rn = 1 
        ) a
    )
    SELECT 
        farm_fingerprint(CONCAT(
            COALESCE(CAST(calday AS STRING), ""),
            COALESCE(CAST(do_number AS STRING), ""),
            COALESCE(CAST(dc_id AS STRING), ""),
            COALESCE(CAST(store_id AS STRING), ""),
            COALESCE(CAST(plan_gi_date AS STRING), ""),
            COALESCE(CAST(actual_gi_date AS STRING), "")
        )) AS hash_id,
        * EXCEPT (rn)
    FROM main 
    WHERE rn = 1
) temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN INSERT (
    hash_id,
    calday,
    do_number,     
    dc_id,     
    store_id,        
    plan_gi_date,        
    actual_gi_date,     
    proc_date,
    file_creation_ts
)
VALUES (
    temp_tbl.hash_id,
    temp_tbl.calday,
    temp_tbl.do_number,
    temp_tbl.dc_id,
    temp_tbl.store_id,
    temp_tbl.plan_gi_date,
    temp_tbl.actual_gi_date,
    temp_tbl.proc_date,
    temp_tbl.file_creation_ts
)
""")